# Processamento do Anexo C

In [11]:
import pandas as pd
from dotenv import load_dotenv
import os
import re
from src.anexo_c.process_dashboard import process_dashboard
from src.anexo_c.export_pdf import export_pdf
from tqdm.notebook import tqdm

## Configurações

In [12]:
load_dotenv()
processamentos = process_dashboard('PIS')

INPUT_DIR = os.getenv('input_dir')
OUTPUT_DIR = os.getenv('output_dir')
INVALID_DIR = os.getenv('invalid_dir')

DASHBOARD_CSV = os.path.join(INPUT_DIR, "Anexo C", "Diligencia_Contratos.csv")
TEMPLATE_PATH = os.path.join(INPUT_DIR, "Anexo C", "template.xlsx")

VALUE_COLS = ['ValorDebito', 'ValorCredito', 'Movimentacao']
GROUP_COLS = ['Conta_Nome','Cosif_Nome']

CONFIGS = [
    (['Total das Receitas', ' Exclusão'], '(01) Total das Receitas'),
    ([' Exclusão'], '(02) Exclusão'),
    ([' Dedução'], '(03) Dedução'),
    (['Sem efeito'],'Sem efeito no Pis Cofins'),
]

## Geração do Excel

In [13]:
dashboard = pd.read_csv(DASHBOARD_CSV, sep=';', encoding='latin1')

In [14]:
contratos = dashboard['NumContrato'].unique().tolist()
#contratos_selecionados = pd.read_csv(r"100 Kits_KPMG.csv")['Contrato'].tolist()
#contratos_to_process = [c for c in contratos if c in contratos_selecionados]
contratos_to_process = contratos

In [15]:
contratos_to_process

[186490,
 221458,
 210515,
 174820,
 185670,
 187993,
 213400,
 280466,
 316952,
 352546,
 358397,
 403642,
 406628,
 410807,
 411310,
 435252,
 463561,
 746287,
 481676,
 494831,
 570927,
 580295,
 597407,
 647286,
 651870,
 653722,
 660421,
 670267,
 674521,
 706264,
 706286,
 723045,
 4573725,
 730621,
 746926,
 689876,
 6306,
 14227,
 35923,
 913357,
 74509,
 97740,
 107213,
 146540,
 150644,
 150889,
 165294,
 300870,
 301504,
 304610,
 343237,
 505314,
 513672,
 519977,
 524927,
 528289,
 532740,
 539410,
 541662,
 541886,
 543683,
 543833,
 547653,
 794680,
 546280,
 555094,
 563365,
 564309,
 779540,
 781654,
 796795,
 805969,
 813322,
 814234,
 816300,
 817018,
 827574,
 833045,
 837741,
 877204,
 846928,
 852133,
 856388,
 856619,
 861395,
 862196,
 866396,
 873040,
 873727,
 874557,
 875696,
 875865,
 888284,
 891851,
 760135,
 907409,
 911245,
 915874,
 916249,
 917280,
 919553,
 927141,
 943824,
 944965,
 960476,
 154561,
 174951,
 175856,
 176774,
 199647,
 203172,
 22165

In [16]:
processamentos.get_contract_numbers(OUTPUT_DIR)

✅ Extraídos 16516 itens e salvos em 'numeros_extraidos.txt'


In [17]:
nao_encontrados_3 = set()
if os.path.isfile("numeros_extraidos.txt"):
    with open("numeros_extraidos.txt", "r") as f:
        nao_encontrados_3 = set(line.strip() for line in f)

# Filtre os contratos removendo aqueles que estão na lista de não encontrados
contratos = [contrato for contrato in contratos_to_process if str(contrato) not in nao_encontrados_3]

for contrato in tqdm(contratos, unit="contrato"):
    tqdm.desc = f"processando contrato: {contrato}"
    dashboard_filtro = dashboard[(dashboard['NumContrato'] == contrato)]


    dashboard_filtro = processamentos.replicate_years(dashboard_filtro,
                                                                group_cols=GROUP_COLS,
                                                                value_cols=VALUE_COLS,
                                                                fill_value=0)
    final = pd.DataFrame()

# para cada filtro, gera o bloco e concatena
    for filtro, desc in CONFIGS:
        bloco = processamentos.group_revenues(dashboard_filtro, filtro, desc)
        final = pd.concat([final, bloco], ignore_index=True)



    df_pis_cofins = processamentos.calcula_pis_cofins(final)
]


    final = pd.concat([final, df_pis_cofins], ignore_index=True)


    final = final[['Ano', 'Descrição', 'Conta_Nome', 'Cosif_Nome', 'ValorCredito', 'Movimentacao']]

    df = final.copy()
    output_path   = os.path.join(OUTPUT_DIR, f"C{contrato}.xlsx")

    processamentos.atualizar_template_pivot(
        template_path=TEMPLATE_PATH,
        output_path=output_path,
        df=df,
        contrato=contrato
    )


  0%|          | 0/6365 [00:00<?, ?contrato/s]

c:\projetos\superveniencia_piscofins\src\anexo_c\process_dashboard.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, self.col_ano] = df[self.col_AnoMes] // 100
c:\projetos\superveniencia_piscofins\src\anexo_c\process_dashboard.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, self.col_ano] = df[self.col_AnoMes] // 100
c:\projetos\superveniencia_piscofins\src\anexo_c\process_dashboard.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                nao_encontrados_3 = set()
if os.path.isfile("numeros_extraidos.txt"):
    with open("numeros_extraidos.txt", "r") as f:
        nao_encontrados_3 = set(line.strip() for line in f)

# Filtre os contratos removendo aqueles que estão na lista de não encontrados
contratos = [contrato for contrato in contratos_to_process if str(contrato) not in nao_encontrados_3]

for contrato in tqdm(contratos, unit="contrato"):
    tqdm.desc = f"processando contrato: {contrato}"
    dashboard_filtro = dashboard[(dashboard['NumContrato'] == contrato)]


    dashboard_filtro = processamentos.replicate_years(dashboard_filtro,
                                                                group_cols=GROUP_COLS,
                                                                value_cols=VALUE_COLS,
                                                                fill_value=0)
    final = pd.DataFrame()

# para cada filtro, gera o bloco e concatena
    for filtro, desc in CONFIGS:
        bloco = processamentos.group_revenues(dashboard_filtro, filtro, desc)
        final = pd.concat([final, bloco], ignore_index=True)



    df_pis_cofins = processamentos.calcula_pis_cofins(final)                                                                                                                                                                                                                                                                                                                                                                                                                                                        



    final = pd.concat([final, df_pis_cofins], ignore_index=True)


    final = final[['Ano', 'Descrição', 'Conta_Nome', 'Cosif_Nome', 'ValorCredito', 'Movimentacao']]

    df = final.copy()
    output_path   = os.path.join(OUTPUT_DIR, f"C{contrato}.xlsx")

    processamentos.atualizar_template_pivot(
        template_path=TEMPLATE_PATH,
        output_path=output_path,
        df=df,
        contrato=contrato
    )


## Exportação do PDF

In [18]:
#export_pdf().process_file()

# ( •̀ ω •́ )y